In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2022-06-25 04:55:52--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2022-06-25 04:55:52--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdb233fce2c0af253a3a834e45b.dl.dropboxusercontent.com/cd/0/inline/Bn0hx_-_sJR6Ivuh2PyuEtWc_VuEGhDxJessHstEBxmW24W0MS03IDE_vV9cncjDe3eaiHa0pdlHvGlPfBHYqR8zWciySMyas-4fG9AJvBNkUUwDTcWCVJrlAqQTtuC0BlpowsHDXnFSNTlzr1NfMOJ5ZLTeEJGb2Wol6uvgiTjSHw/file# [following]
--2022-06-25 04:55:52--  https://ucdb233fce2c0af253a3a834e45b.dl.dropboxusercontent.com/cd/0/inline/Bn0hx_-_sJR6Ivuh2PyuEtWc_VuEG

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

replace movie-reviews-dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
# Sentiment Analysis

In [4]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, RNN, SimpleRNNCell, Embedding, Dropout

In [5]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
      lambda text, label: (regex_replace(text, '<br />', ' '), label)
  )

In [6]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b"Here it is.. the first EVER episode of Friends, Where we get introduced to Control Freak Monica Gellar (Courtney Cox), Newly divorced Ross Gellar (David Schimmer), Hippy Pheobe Buffay (Lisa Kudrow), unknown actor and ladies man (Matt Le Blanc and very sarcastic Chandler Bing (Matthew Perry). This is how the scene starts off until we introduced to the 6th and final friend Spoilt kid Rachel Green (Jennifer Aniston).  The Episode is better than most people give credit for, like any new sitcom the first episode isn't always fantastic. The acting in this episode isn't great because the cast cannot identify and arnt really believable in their new characters (apart from Kudrow and Perry- who shine).  Matt Le Blanc- Man, his acting was down right dreadful because until later, he gets more confident, but i think he tries to be funny but at most fails.  David Schimmer- Why does he over pronounce EVERY word? he 

In [7]:
model = Sequential()

In [8]:
model.add(Input(shape=(1,), dtype="string"))

In [9]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_len
)

In [10]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [11]:
model.add(Embedding(max_tokens + 1, 128))

# if you need more RNNCells set return_state to True.
# We don't need the hidden state why we set return_state to False.

rnn = RNN(SimpleRNNCell(64), return_sequences=False, return_state=False)
model.add(rnn)
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [12]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 38s 47ms/step - loss: 0.6977 - accuracy: 0.5104
Epoch 2/10
782/782 [==============================] - 37s 47ms/step - loss: 0.6875 - accuracy: 0.5446
Epoch 3/10
782/782 [==============================] - 36s 45ms/step - loss: 0.6755 - accuracy: 0.5746
Epoch 4/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6497 - accuracy: 0.6232
Epoch 5/10
782/782 [==============================] - 37s 47ms/step - loss: 0.6575 - accuracy: 0.6070
Epoch 6/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6584 - accuracy: 0.5985
Epoch 7/10
782/782 [==============================] - 36s 46ms/step - loss: 0.6380 - accuracy: 0.6330
Epoch 8/10
782/782 [==============================] - 37s 47ms/step - loss: 0.6215 - accuracy: 0.6528
Epoch 9/10
782/782 [==============================] - 36s 46ms/step - loss: 0.5861 - accuracy: 0.6913
Epoch 10/10
782/782 [==============================] - 36s 46ms/step - loss: 0.570

In [14]:
model.evaluate(test_data)

782/782 [==============================] - 17s 21ms/step - loss: 0.6372 - accuracy: 0.6570


[0.6372220516204834, 0.657039999961853]

In [15]:
text = "I loved the movie !"

In [16]:
model.predict([text])

array([[0.7431413]], dtype=float32)